In [315]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd")

In [316]:
#Read the data in
nyc_crime_raw = pd.read_csv('nyc_crime_2014.csv')

### Describe the Data

In [317]:
#Take a quick glance at the data
nyc_crime_raw.head(5)

,City,Population,ViolentCrime,Murder,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,Unnamed: 12
0,Adams Village,"1,851",0,0.000,NaN,0,0,11,1,10,0,0.000,nan
1,Addison Town and Village,"2,568",2,0.000,NaN,1,1,49,1,47,1,0.000,nan
2,Afton Village4,820,0,0.000,0,0,0,1,0,1,0,0.000,nan
3,Akron Village,"2,842",1,0.000,NaN,0,1,17,0,17,0,0.000,nan
4,Albany4,"98,595",802,8.000,54,237,503,"3,888",683,"3,083",122,12.000,nan


In [318]:
#Remove superfluous column
nyc_crime_raw.drop(columns=['Unnamed: 12'], inplace=True)

In [319]:
nyc_crime_raw.shape

(376, 12)

In [320]:
nyc_crime_raw.describe()

,Murder,Arson
count,369.000,365.000
mean,1.453,1.425
std,17.694,7.995
min,0.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,1.000
max,333.000,135.000


### Clean the Data (remove strings, NaN's, and convert to int)

In [321]:
#Now that we know we have 376 observations across 12 columns, let's Find NaN's by column

missing_values_count = nyc_crime_raw.isnull().sum()
print(missing_values_count)

City                   1
Population             7
ViolentCrime           7
Murder                 7
Rape                 149
Robbery                7
AggravatedAssault      7
PropertyCrime          8
Burglary               7
LarcenyTheft           8
MotorVehicleTheft      7
Arson                 11
dtype: int64


**Drop NaN's in the columns we care most about**

In [322]:
nyc_crime_raw = nyc_crime_raw.dropna(subset=['Population', 'Robbery', 'Burglary', 'PropertyCrime'])

**Remove commas from all columns of interest**

In [323]:
nyc_crime_raw['Population'] = nyc_crime_raw['Population'].apply(lambda x: str(x).replace(',', ''))

In [324]:
nyc_crime_raw['Robbery'] = nyc_crime_raw['Robbery'].apply(lambda x: str(x).replace(',', ''))

In [325]:
nyc_crime_raw['Burglary'] = nyc_crime_raw['Burglary'].apply(lambda x: str(x).replace(',', ''))

In [326]:
nyc_crime_raw['PropertyCrime'] = nyc_crime_raw['PropertyCrime'].apply(lambda x: str(x).replace(',', ''))

**Change columns to int**

In [327]:
nyc_crime_raw['Population'] = nyc_crime_raw['Population'].astype(int)

In [328]:
nyc_crime_raw['Robbery'] = nyc_crime_raw['Robbery'].astype(int)

In [329]:
nyc_crime_raw['Burglary'] = nyc_crime_raw['Burglary'].astype(int)

In [330]:
nyc_crime_raw['PropertyCrime'] = nyc_crime_raw['PropertyCrime'].astype(int)

### Purge outliers and potential skewness

In [331]:
print(nyc_crime_raw['PropertyCrime'].describe())
print(nyc_crime_raw['Population'].describe())
print(nyc_crime_raw['Robbery'].describe())
print(nyc_crime_raw['Burglary'].describe())

count      368.000
mean       698.361
std       7123.614
min          0.000
25%         25.000
50%         76.000
75%        271.500
max     135747.000
Name: PropertyCrime, dtype: float64
count       368.000
mean      37888.399
std      441757.416
min          79.000
25%        2628.250
50%        6564.500
75%       15534.750
max     8473938.000
Name: Population, dtype: float64
count     368.000
mean       60.823
std       867.655
min         0.000
25%         0.000
50%         1.000
75%         4.000
max     16581.000
Name: Robbery, dtype: float64
count     368.000
mean      101.160
std       856.253
min         0.000
25%         4.000
50%        12.500
75%        39.000
max     15916.000
Name: Burglary, dtype: float64


**Start by removing data less than the 25th percentile, then removing extreme outliers by using the quantile() method. Do this for all variables.

In [332]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['PropertyCrime'] > 25]

In [334]:
nyc_crime_raw['PropertyCrime'].quantile(0.95)

2009.4499999999998

In [335]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['PropertyCrime'] < 2009]

In [336]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Population'] > 2628]

In [338]:
nyc_crime_raw['Population'].quantile(0.95)

44738.79999999999

In [339]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Population'] < 66486]

In [340]:
#Robbery is different. The 25th percentile is 0. Let's instead look at a lower quantile
nyc_crime_raw['Robbery'].quantile(0.85)

12.0

In [341]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Robbery'] < 7]

In [342]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Burglary'] > 4]

In [343]:
nyc_crime_raw['Burglary'].quantile(0.95)

61.19999999999999

In [344]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Burglary'] < 61]

**Create Population Squared Feature**

In [345]:
nyc_crime_raw['PopulationSquared'] = nyc_crime_raw['Population']**2

### Run the model

In [346]:
# Instantiate and fit our model.
regression = linear_model.LinearRegression()
Y = nyc_crime_raw['PropertyCrime']
X = nyc_crime_raw[['Population', 'PopulationSquared', 'Burglary', 'Robbery']]
regression.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regression.coef_)
print('\nIntercept: \n', regression.intercept_)
print('\nR-squared:')
print(regression.score(X, Y))


Coefficients: 
 [3.34703039e-03 2.82315487e-08 4.87213440e+00 5.95029612e+00]

Intercept: 
 -5.231039211108111

R-squared:
0.6341975248011626


### Validate the model

In [347]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(regression, X, Y, cv=10)

array([ 0.71918411,  0.85044808,  0.25302344,  0.36140108,  0.584955  ,
        0.79021503,  0.81217316, -0.02479547,  0.58203908,  0.51212118])

In [348]:
# Use a ~ to represent an '=' from the functional form
linear_formula = 'PropertyCrime ~ Population+PopulationSquared+Burglary+Robbery'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=nyc_crime_raw).fit()

In [349]:
lm.params

Intercept           -5.231
Population           0.003
PopulationSquared    0.000
Burglary             4.872
Robbery              5.950
dtype: float64

In [350]:
lm.pvalues

Intercept           0.754
Population          0.211
PopulationSquared   0.712
Burglary            0.000
Robbery             0.133
dtype: float64

In [351]:
lm.rsquared

0.6341975248011626

In [352]:
lm.conf_int()

,0,1
Intercept,-38.202,27.740
Population,-0.002,0.009
PopulationSquared,-0.000,0.000
Burglary,3.844,5.901
Robbery,-1.842,13.742


### Create revised model